## Preprocessing (some changes made)

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#Optimization: removed a few more columns
application_df=application_df.drop(['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS', 'INCOME_AMT'], axis=1)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,6692,1
4,T3,Independent,C1000,Heathcare,Trust,142590,1
...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,5000,0
34297,T5,Independent,C3000,ProductDev,Association,5000,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_count = application_df["CLASSIFICATION"].value_counts()
filtered_class_count = class_count[class_count > 1]
filtered_class_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classes = application_df['CLASSIFICATION'].unique()
useful_classes = ['C1000', 'C2000', 'C1200', 'C3000', 'C2100']
classifications_to_replace = np.delete(classes, np.where(np.isin(classes, useful_classes)))

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies = pd.get_dummies(application_df.loc[:, application_df.columns != "IS_SUCCESSFUL"])

In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
#X = application_df.drop(["IS_SUCCESSFUL"],axis=1).values
X = dummies
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Optimization Attempt 1

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# OPTIMIZATION Attempt 1 - increasing the neurons in each layer and add three more hidden layers
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  400
hidden_nodes_layer2 = 300
hidden_nodes_layer3 = 200
hidden_nodes_layer4 = 100
hidden_nodes_layer5 = 50
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 400)               12800     
                                                                 
 dense_1 (Dense)             (None, 300)               120300    
                                                                 
 dense_2 (Dense)             (None, 200)               60200     
                                                                 
 dense_3 (Dense)             (None, 200)               40200     
                                                                 
 dense_4 (Dense)             (None, 200)               40200     
                                                                 
 dense_5 (Dense)             (None, 1)                 201       
                                                                 
Total params: 273,901
Trainable params: 273,901
Non-trai

In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
# Optimization - increase the number of epochs
fit_model = nn.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
804/804 [==============================] - 17s 17ms/step - loss: 0.5760 - accuracy: 0.7204
Epoch 2/200
804/804 [==============================] - 10s 13ms/step - loss: 0.5663 - accuracy: 0.7249
Epoch 3/200
804/804 [==============================] - 7s 8ms/step - loss: 0.5621 - accuracy: 0.7262
Epoch 4/200
804/804 [==============================] - 8s 9ms/step - loss: 0.5615 - accuracy: 0.7265
Epoch 5/200
804/804 [==============================] - 7s 9ms/step - loss: 0.5607 - accuracy: 0.7264
Epoch 6/200
804/804 [==============================] - 7s 9ms/step - loss: 0.5606 - accuracy: 0.7281
Epoch 7/200
804/804 [==============================] - 8s 10ms/step - loss: 0.5590 - accuracy: 0.7272
Epoch 8/200
804/804 [==============================] - 6s 8ms/step - loss: 0.5587 - accuracy: 0.7285
Epoch 9/200
804/804 [==============================] - 8s 10ms/step - loss: 0.5572 - accuracy: 0.7296
Epoch 10/200
804/804 [==============================] - 6s 8ms/step - loss: 0.5576 - 

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6120 - accuracy: 0.7271 - 1s/epoch - 4ms/step
Loss: 0.6119717955589294, Accuracy: 0.7271137237548828


In [17]:
# Export our model to HDF5 file
nn.save_weights("AlphabetSoupCharity_Optimized.h5")

## Optimization Attempt 2

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# OPTIMIZATION Attempt 2 - change the number of layers and change the activation layer for one of the hidden layers
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  200
hidden_nodes_layer2 = 120
hidden_nodes_layer3 = 50
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 200)               6400      
                                                                 
 dense_9 (Dense)             (None, 120)               24120     
                                                                 
 dense_10 (Dense)            (None, 50)                6050      
                                                                 
 dense_11 (Dense)            (None, 1)                 51        
                                                                 
Total params: 36,621
Trainable params: 36,621
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5766 - accuracy: 0.7168
Epoch 2/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5644 - accuracy: 0.7252
Epoch 3/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5625 - accuracy: 0.7256
Epoch 4/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5605 - accuracy: 0.7277
Epoch 5/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5601 - accuracy: 0.7275
Epoch 6/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5589 - accuracy: 0.7286
Epoch 7/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5579 - accuracy: 0.7289
Epoch 8/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5577 - accuracy: 0.7278
Epoch 9/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5568 - accuracy: 0.7290
Epoch 10/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5571 - accura

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5585 - accuracy: 0.7289 - 575ms/epoch - 2ms/step
Loss: 0.5584551095962524, Accuracy: 0.728863000869751


In [23]:
# Export our model to HDF5 file
nn.save_weights("AlphabetSoupCharity_Optimized2.h5")

## Optimization Attempt 3

In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# OPTIMIZATION Attempt 3 - change the number of layers, number of neurons, and change the activation layer for more of the hidden layers
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  400
hidden_nodes_layer2 = 300
hidden_nodes_layer3 = 200
hidden_nodes_layer4 = 80
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="sigmoid"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 400)               12800     
                                                                 
 dense_23 (Dense)            (None, 300)               120300    
                                                                 
 dense_24 (Dense)            (None, 200)               60200     
                                                                 
 dense_25 (Dense)            (None, 200)               40200     
                                                                 
 dense_26 (Dense)            (None, 1)                 201       
                                                                 
Total params: 233,701
Trainable params: 233,701
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [31]:
# Train the model
# Optimization - increase the number of epochs even more
fit_model = nn.fit(X_train_scaled,y_train,epochs=300)

Epoch 1/300
804/804 [==============================] - 8s 9ms/step - loss: 0.5982 - accuracy: 0.7014
Epoch 2/300
804/804 [==============================] - 6s 7ms/step - loss: 0.5737 - accuracy: 0.7216
Epoch 3/300
804/804 [==============================] - 7s 8ms/step - loss: 0.5684 - accuracy: 0.7252
Epoch 4/300
804/804 [==============================] - 6s 8ms/step - loss: 0.5655 - accuracy: 0.7241
Epoch 5/300
804/804 [==============================] - 6s 8ms/step - loss: 0.5635 - accuracy: 0.7255
Epoch 6/300
804/804 [==============================] - 7s 8ms/step - loss: 0.5631 - accuracy: 0.7251
Epoch 7/300
804/804 [==============================] - 6s 8ms/step - loss: 0.5625 - accuracy: 0.7257
Epoch 8/300
804/804 [==============================] - 7s 8ms/step - loss: 0.5619 - accuracy: 0.7266
Epoch 9/300
804/804 [==============================] - 6s 8ms/step - loss: 0.5617 - accuracy: 0.7242
Epoch 10/300
804/804 [==============================] - 7s 8ms/step - loss: 0.5614 - accura

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5672 - accuracy: 0.7272 - 593ms/epoch - 2ms/step
Loss: 0.567150890827179, Accuracy: 0.7272303104400635


In [32]:
# Export our model to HDF5 file
nn.save_weights("AlphabetSoupCharity_Optimized3.h5")